In [1]:
import gammapy 
print(f'loaded gammapy version: {gammapy.__version__} ' )
print(f'Supposed to be 1.0 (21-12-2022)' )

loaded gammapy version: 0.1.dev19174+g56b5bdf 
Supposed to be 1.0 (21-12-2022)


In [2]:
#get_ipython().system('jupyter nbconvert --to script 1-Nui_Par_Fitting.ipynb')
import pyximport

pyximport.install()
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
import gammapy

# from gammapy.datasets import MapDataset
from gammapy.maps import Map
from astropy.coordinates import SkyCoord, Angle
from gammapy.modeling import Fit, Parameter, Parameters, Covariance
from gammapy.datasets import MapDataset #, MapDatasetNuisance
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    create_crab_spectral_model,
    SkyModel,
    PointSpatialModel,
    ShellSpatialModel,
    GeneralizedGaussianSpatialModel,
    TemplateSpatialModel,
    LogParabolaSpectralModel,
    GaussianSpatialModel,
    DiskSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    SpatialModel,
    FoVBackgroundModel,
)
from gammapy.estimators import TSMapEstimator, ExcessMapEstimator
from gammapy.maps import MapAxis
from gammapy.modeling.models.spectral import scale_plot_flux
from gammapy.estimators import  FluxPointsEstimator
    
from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys

from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys
sys.path.append('/home/katrin/Documents/nuisance_summary/')
sys.path.append('../')
from Dataset_Creation import sys_dataset
import Dataset_load 

## Read dataset

In [3]:
scaled_amplitude = Parameter('amplitude', 
                             value = 1e-12)
cutoff = 60
cutoff_input_value = cutoff
lambda_ = Parameter('lambda_', value = 1/ cutoff )
dataset_asimov  = Dataset_load.create_asimov(model = 'ecpl',source = "PKSflare",
                                            parameters = Parameters([scaled_amplitude, lambda_]))


loaded dataset:
/home/wecapstor1/caph/mppi045h/nuisance_summary/PKS_flare/HESS_public/dataset-simulated-ecpl.fits.gz


## Varying the Exposure

In [4]:
mask = dataset_asimov.mask.data.sum(axis=2).sum(axis=1)>0

ebins = dataset_asimov.counts.geom.axes[0].center[mask]
print(len(ebins))

20


In [5]:
bias_assumed = 0.1
sigma_assumed = 0.1



N = 300

save = True
save_flux = True




In [8]:
import os 
path = os.getcwd() +"/Bias/Cutoff/"
print(path)

/home/wecapstor1/caph/mppi045h/nuisance_summary/E_reco_PKS_flare/Bias/Cutoff/


In [ ]:
for n in range(N):
    print()
    print('====' * 30)
    print(n)
    print('====' * 30)
    
    bias_rnd = np.random.normal(0, bias_assumed, 1)
    sigma_rnd = np.random.normal(0, sigma_assumed, 1)
    print(f"bias:, {bias_rnd}, res: {sigma_rnd}" )
    sys_d_cor = sys_dataset(dataset_asimov= dataset_asimov,
                    shift = 0,
                        tilt = 0,
                        bias = bias_rnd, 
                        resolution = sigma_rnd,
                    rnd = True)
    dataset = sys_d_cor.create_dataset()
    fit_cor = Fit(store_trace=False)
    minuit_opts = {"tol": 0.001, "strategy": 2}
    fit_cor.optimize_opts = minuit_opts
    result_cor = fit_cor.run([dataset])


    if save:
        with open(path+"data/7a_P_draw_info.txt", "a") as myfile:
            myfile.write(str(float(bias_rnd)) + '    '+ str(float(sigma_rnd)) + '    ' +  str(float(dataset.stat_sum())) + '\n')

    stri = ""
    for p in ['lambda_', 'index', 'norm', 'tilt']:
        stri += str(dataset.models.parameters[p].value)  + '   ' +  str(dataset.models.parameters[p].error)  + '   '
    print(stri)
    if save:
        with open(path+"data/7a_P_draw_par.txt", "a") as myfile:
            myfile.write(stri + '\n')



    fluxes = []
    for e in ebins:
        flux =  dataset.models[0].spectral_model(e)
        fluxes.append(flux.value)

    ff = str()
    for f in fluxes:
        ff += str(f) + "  "
    #print(ff)
    if save:
        with open(path+"data/7a_P_draw_flux.txt", "a") as myfile:
            myfile.write( ff + '\n')


    energy_bounds = (ebins[0], ebins[-1] ) * u.TeV

    energy_min, energy_max = energy_bounds
    energy = MapAxis.from_energy_bounds(
        energy_min,
        energy_max,
        len(ebins),
    )

    fluxe2, _ = dataset.models[0].spectral_model._get_plot_flux(sed_type='dnde', energy=energy)
    fluxe2 = scale_plot_flux(fluxe2, energy_power=2)
    fluxe2 = fluxe2.quantity[:, 0, 0]
    fluxe2 = np.array(fluxe2)   
    ff = str()
    for f in fluxe2:
        ff += str(f) + "  "
    if save:
        with open(path+"data/7a_P_draw_flux2e.txt", "a") as myfile:
            myfile.write( ff + '\n')

    energy_edges = dataset.geoms['geom'].axes[0].edges
    #esti  = FluxPointsEstimator(energy_edges= energy_edges)
    #fluxpoints = esti.run([dataset])
    #if save_flux:
    #    fluxpoints.write(f'data/fluxpoints/6_fluxpoints_{shift_rnd[0]:.6}.fits')
    #except:
    #    pass



0
bias:, [-0.07322567], res: [-0.03904066]
0.036353553398307294   0.015440314260539137   2.0543887457814334   0.07683055322661805   1.0009707738437257   0.0015609872883788848   0.0   0.0   

1
bias:, [-0.01634717], res: [0.03522855]
0.031514875527681126   0.0190175778042759   2.330946909994454   0.07650998310046293   1.0006891608835184   0.0015609181330474764   0.0   0.0   

2
bias:, [-0.00474514], res: [0.07245417]
0.02179365571772131   0.015722077856351938   2.2614316152386342   0.07718899680360644   0.9975108063182908   0.00155830773999758   0.0   0.0   

3
bias:, [0.13064796], res: [-0.09871111]
0.015308290643888814   0.014224215858798524   2.326406358487998   0.07395250282288046   0.9996947176695898   0.0015600609972258527   0.0   0.0   

4
bias:, [0.03854478], res: [0.1249623]
0.02172275310617277   0.01466999271149718   2.2041481911679592   0.08059643018679152   0.9964099404605461   0.0015573414352749598   0.0   0.0   

5
bias:, [-0.02537372], res: [-0.07930901]
0.06458845144011